##Installation of the libraries

In [1]:
!pip install -q transformers

In [2]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.9 MB/s eta 0:00:00


In [3]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Torch optimization.

##All libraries needed for training

In [4]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting rid of the sequences which are outside of the threshold (64-128).

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [13]:
file_path = "AF-A0A1D8PD42-F1-model_v4.cif"
file_model = "AF-A0A1D8PD42-F1-model_v4"
pdbl = PDBList()
#pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

MSSSNTDNQYPKYINDTTPPTITLKEYDNASWASTTCLDHNPIKNQYIVVVMENPNQIVAIIDQQDNMILDILFKNAHDAHSKQEYSTK


Calculating the angles for the given sequence

In [15]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

98.16395945950967
82.50495828587032
79.19914435971504
83.07405176142494
60.044313470532636
102.7946554274031
85.28666894875717
42.93453812073845
127.9656719708327
155.69465179650248
138.024605537606
137.14954131146746
146.56270183468035
-34.044110750278094
-179.8297162210937
-4.496276224371437
-13.865314978627604
110.50093422515786
148.99421230816424
154.43011759933302
130.1698008731835
157.84142541088352
157.48084112080286
-35.68161727313367
-41.29169954519074
-40.064017274326616
-50.02143004729109
-27.961036819301654
-0.5788015082387981
132.7397194649513
-31.34336919414818
-4.1441287979794685
-30.813828227805875
-11.546966123488676
5.479622169341699
166.80301504721157
156.04470680565288
135.83116383955326
159.44668064821525
123.83160173972804
105.78580725491983
-25.963550151453397
-41.55782497300403
-4.671401085177566
54.0435532210131
151.3552320878784
134.9018875649975
155.74597636554063
125.43791323308822
154.7034618100038
128.30647901386703
-39.96575503862786
-19.282738126548896
8

Putting angles in a matrix.

In [16]:
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [17]:
seq_example =  ' '.join(sequence)
seq_example

'M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K'

##Embedding space creation (using Prot-Bert)

In [10]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

Getting the dimensions of the embedding space.

In [18]:
N , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]
embedding_prot_bert = output.last_hidden_state


NameError: name 'output' is not defined

In [12]:
print(N)
print(D)

NameError: name 'N' is not defined

##Single-head self unmasked attention layer

In [20]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [21]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.ReLU(),
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.ReLU(),
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return output


In [22]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.05)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            self.optimizer.zero_grad()
            #self.scheduler.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")




In [23]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 200

model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
model.forward(embeddings)
criterion = nn.MSELoss()
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Epoch 1/200, Loss: 6.061740875244141
Epoch 2/200, Loss: 5.774256229400635
Epoch 3/200, Loss: 5.540521621704102
Epoch 4/200, Loss: 6.361536026000977
Epoch 5/200, Loss: 7.388876914978027
Epoch 6/200, Loss: 5.711643695831299
Epoch 7/200, Loss: 5.197005271911621
Epoch 8/200, Loss: 5.073453903198242
Epoch 9/200, Loss: 4.9733781814575195
Epoch 10/200, Loss: 4.892316818237305
Epoch 11/200, Loss: 4.826656818389893
Epoch 12/200, Loss: 4.773471832275391
Epoch 13/200, Loss: 4.7303924560546875
Epoch 14/200, Loss: 4.695497989654541
Epoch 15/200, Loss: 4.667233467102051
Epoch 16/200, Loss: 4.644339084625244
Epoch 17/200, Loss: 4.625794887542725
Epoch 18/200, Loss: 4.610774040222168
Epoch 19/200, Loss: 4.598607063293457
Epoch 20/200, Loss: 4.588751792907715
Epoch 21/200, Loss: 4.5807695388793945
Epoch 22/200, Loss: 4.57430362701416
Epoch 23/200, Loss: 4.569065570831299
Epoch 24/200, Loss: 4.564823627471924
Epoch 25/200, Loss: 4.5613861083984375
Epoch 26/200, Loss: 4.558603286743164
Epoch 27/200, Loss

TransformerModel(
  (self_attention): SelfAttention()
  (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
  (feed_forward): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)

In [24]:
angles_tensor = torch.from_numpy(angles.T)
predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [25]:
pred = predicted_angles*(180/np.pi)
torch.save(pred,'predictions/N_A0A1D8PD42-F1_200_05_SGD.pt')

RuntimeError: Parent directory predictions does not exist.

In [26]:
angles_tensor*(180/np.pi)

tensor([[ 179.9299,   98.1640],
        [ 166.9430,   82.5050],
        [ 175.7644,   79.1991],
        [ 153.2986,   83.0741],
        [ 139.6692,   60.0443],
        [ 176.9491,  102.7947],
        [ 164.3176,   85.2867],
        [ 178.5258,   42.9345],
        [ 171.2007,  127.9657],
        [ 174.5847,  155.6947],
        [ 173.7650,  138.0246],
        [ 172.4886,  137.1495],
        [ 179.3246,  146.5627],
        [ 170.6717,  -34.0441],
        [-176.4155, -179.8297],
        [-173.8806,   -4.4963],
        [-169.4366,  -13.8653],
        [ 173.5445,  110.5009],
        [ 175.3131,  148.9942],
        [ 167.3701,  154.4301],
        [ 176.2569,  130.1698],
        [ 170.4209,  157.8414],
        [ 171.9277,  157.4808],
        [ 175.1485,  -35.6816],
        [ 175.1972,  -41.2917],
        [ 173.9927,  -40.0640],
        [-179.1476,  -50.0214],
        [-179.6617,  -27.9610],
        [-177.6897,   -0.5788],
        [ 176.5145,  132.7397],
        [ 179.8077,  -31.3434],
        

In [28]:
np.shape(angles_tensor)

torch.Size([89, 2])

In [30]:
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('AF-A0A1D8PD42-F1-model_v4_pred.pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering



AF-A0A1D8PD42-F1-model_v4 A coordinates for 707 dihedra updated in 270 iterations
